In [6]:
from __future__ import unicode_literals, print_function
import random
from pathlib import Path
import spacy
from spacy import displacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, compile_infix_regex, compile_suffix_regex,minibatch, compounding
import re
from collections import Counter
import en_core_web_sm
from pprint import pprint
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher
import unidecode 
import string
from datetime import datetime
import docx
from docx import Document
from win32com import client as wc

In [7]:
def convert(path):
    word = wc.Dispatch('Word.Application')
    doc = word.Documents.Open(path)  
    doc.SaveAs(path+'.docx', 12) 
    doc.SaveAs(path+'.txt',4)
    d = Document(path+'.docx') 
    if d.tables!=[]:
        print(d.tables)
    doc.Close()
    word.Quit()
    return path+'.txt'

# d=convert(r'D:\UN\N1906016.DOC')

In [8]:

def read(path):
    path_txt=convert(path)
    file=open(path_txt,'r')
    file=file.read()
    file2=unidecode.unidecode(file)
    return file2

In [9]:
nlp = spacy.load("en_core_web_sm")
infixes = nlp.Defaults.prefixes + ( r"[-]~",r'[/]~',r'\.')
infix_re = spacy.util.compile_infix_regex(infixes)
def custom_tokenizer(nlp):
    return Tokenizer(nlp.vocab, infix_finditer=infix_re.finditer)

nlp.tokenizer = custom_tokenizer(nlp)

In [10]:
# #改地址
# # file=read('N1906016.DOC.txt')
# # file2=read('N0057625.txt')
# # file3=read('N0550446.txt')
# # file4=read('N0147024.txt')
# # file5=read('N0055335.txt')
# file6=read(r'D:\UN\N0550488.txt')
# # file7=read('N0550398.txt')
# file8=read(r'D:\UN\N1821512.txt')
# # doc=nlp(file)
# # doc2=nlp(file2)
# # doc3=nlp(file3)
# # doc4=nlp(file4)
# # doc5=nlp(file5)
# doc6=nlp(file6)
# # doc7=nlp(file7)
# doc8=nlp(file8)

In [11]:
def session(file):
    doc=nlp(file)
    matcher = Matcher(nlp.vocab)
    pattern = [{},{'ORTH': 'session'}, {'IS_SPACE': True}]
    matcher.add('session', None, pattern)
    matches = matcher(doc)
    spans = [(ent_id, doc[start : end-1]) for ent_id, start, end in matches]
    return spans[0][1]

def approval_date(file):
    start=file.find('adopted by the General Assembly on')
    if start!=-1:
        end=file[start:].find('\n')
        item=file[start:start+end]
        date=nlp(item)[6:]
    else:
        start=file.find('plenary meeting')
        start2=file[start:].find('\n')
        item=file[start+start2:]
        item=str(nlp(item)[:4]).strip()
        date=nlp(item)
    d = datetime.strptime(str(date), '%d %B %Y')
    return d.strftime('%Y-%m-%d')

def id_num(file):
    doc=nlp(file)
    pattern=re.compile(r'A/RES/\d+/\d+.+')
    for match in re.finditer(pattern, doc.text):
        start, end = match.span()
        num=doc.text[start:end]
    return num

    
def proponent_authority(file):
    org=[]
    start=file.find('on the report of')
    if start!=-1:
        end=file[start:].find('\n')
        item=file[start:start+end]
        for entity in nlp(item).ents:
            if entity.label_=='ORG':
                org.append(entity.text)
        return ' '.join(org)
    else:
        return 'plenary'   
    
def agenda_item(file):
    start=file.find('Agenda item')
    end=file[start:].find('\n')
    item=file[start:start+end]
    return nlp(item)[2:]

def title(file):
    en=file.find('The General Assembly,')
    doc=nlp(file[:en])
    pattern=re.compile('\d+/\d+\.')
    match=re.search(pattern,doc.text)
    start,end=match.span()
    title=str(list(nlp(doc.text[end:]).sents)[0]).strip()
    return title

def annex(file):
    file=file[:file.rfind('[')]
    start=file.find('Annex')
    if start==-1:
        return None
    else:
        try:
            tfile=file[start:]
            pattern=re.compile('[\t|\n]1(\s)(\t)?[A-Z].+\.')
            match=re.search(pattern, tfile)
            return nlp(tfile[:match.span()[0]])
        except:
            return nlp(file[start:])
    
def preamble(file,nlp=nlp):
    file=file[file.find('The General Assembly,')+len('The General Assembly,'):]
    pattern=re.compile('[A-Z][a-z]+')
    match = pattern.search(file)
    if str(file[match.span()[0]:match.span()[1]])[-1]=='s':
        return None
    else:
        pattern=re.compile('[\s\S]*,(\d+)?(\s+)?\n')
        match=pattern.search(file)
        return nlp(file[:match.span()[1]+1])
    
def operative(file,nlp=nlp):
    file=file[file.find('The General Assembly,')+len('The General Assembly,'):]
    file=file[:file.find('plenary meeting')]
    file=file[:file.rfind('.')+1]

    pattern=re.compile('[^\d]1\.(\s|\t)')
    match=pattern.search(file)
    sep2=match.span()[0]
    if sep2!=-1:
        doc=nlp(file[sep2-1:])
    else:
        pattern=re.compile('[\s\S]*,(\d+)?(\s+)?\n')
        match=pattern.search(file)
        file=file[match.span()[1]:]
        pattern2=re.compile('[A-Z][a-z]+s')
        match=pattern.search(file)
        doc=nlp(file[match.span()[0]:])
    return doc

    
def closing_formula(file):
    mark=file.find('plenary meeting')
    start=file[:mark].rfind('\n')
    doc=nlp(file[start+1:])
    matcher = Matcher(nlp.vocab)
    pattern = [{'IS_DIGIT':True},{'IS_TITLE':True}, {'IS_DIGIT': True}]
    matcher.add('date', None, pattern)
    matches = matcher(doc)
    spans = [(ent_id,doc[:end]) for ent_id, start, end in matches][0][1]
    return spans
    
    
             
def footnote(file):   # ** unrecognize file 4
    file=file[:file.rfind('[')]
    file=file[file.find('plenary meeting'):]
    doc=nlp(file)
    matcher = Matcher(nlp.vocab)
    pattern = [{'IS_DIGIT':True},{'IS_TITLE':True}, {'IS_DIGIT': True}]
    matcher.add('date', None, pattern)
    matches = matcher(doc)
    spans = [(ent_id,doc [end+1:]) for ent_id, start, end in matches][0][1]
    start=str(spans).find('Annex')
    if start==-1:
        return spans
    else:
        try:
            pattern=re.compile('[\t|\n]1(\s)(\t)?[A-Z].+\.') ##file8 
            match=pattern.search(file)
            return nlp(file[match.span()[0]:])
        except:
            return None



In [12]:
def reference(file): 
    reference=[]
    doc=preamble(file)
    matcher = Matcher(nlp.vocab)
    pattern=[{'ORTH':{'IN':['resolution','resolutions','decision','decisions']}},
             {'TEXT':{'REGEX':'(\d+/\d+\s[A-Z]\s)|([A-Z]+/\d+)|(\d+/\d+)|(\d+)'}},
             {'ORTH':{'IN':['A','B','C','D','E']},'OP':'?'}]
    matcher.add('citation',None, pattern)
    matches = matcher(doc)
    spans = [(start,(doc[start:end])) for ent_id, start, end in matches]
    for (start,i) in spans:
        if str(i[0])=='resolutions' or str(i[0])=='decisions':
            pattern2=re.compile(',([^a-zA-Z]+)?\n')
            end=pattern2.search(doc[start+1:].text).span()[1]
            doc2=(doc[start+1:].text)[:end]
            an_re=doc2.find('resolution')
            an_de=doc2.find('decision')
            if an_re==-1 and an_de==-1:
                pass
            elif an_re==-1 or an_de==-1:
                doc2=doc2[:max(an_re,an_de)]
            else:
                doc2=doc2[:min(an_re,an_de)]
            cites=[]
            found=re.findall(re.compile('(\d+/\d+\s[A-Z]\s)|([A-Z]+/\d+)|(\d+/\d+)|(\d+[^a-z\n,]+[(])'),doc2)
            for w in found:
                c=[i for i in w if str(i)!=''] 
                if c[0][-1]=='(':
                    c[0]=c[0][:-1]
                cites.append(c[0])
            cite=str(i[0])+' '+' '.join(cites)
            reference.append(nlp(cite))
        else:
            reference.append(i)
    return reference

In [13]:
def places(file):   ##x 6 aff need trainig
    pre_loc,op_loc=[],[]
    pre=preamble(file)
    for entity in pre.ents:
        if entity.label_=='GPE':
            print(entity.text)
            if entity.text[0].isdigit() or entity.text=='States':
                continue
            text=entity.text.rstrip(string.digits)
            pre_loc.append(text)
    op=operative(file)
    for entity in op.ents:
        if entity.label_=='GPE':
            if entity.text[0].isdigit() or entity.text=='States':
                continue
            text=entity.text.rstrip(string.digits)
            op_loc.append(entity.text)
    return pre_loc,op_loc

In [14]:
def org(file,nlp=nlp):   
    pre_loc,op_loc=[],[]
    pre=preamble(file,nlp)
    for entity in pre.ents:
        if entity.label_=='ORG':
            if entity.text[0].isdigit() or entity.text=='States':
                continue
            text=entity.text.rstrip(string.digits)
            pre_loc.append(text)
    op=operative(file,nlp)
    for entity in op.ents:
        if entity.label_=='ORG':
            if entity.text[0].isdigit() or entity.text=='States':
                continue
            text=entity.text.rstrip(string.digits)
            op_loc.append(entity.text)
    return pre_loc,op_loc

In [15]:
def future_date(file,nlp=nlp):   ##再看看
    approval=approval_date(file)
    doc=operative(file,nlp)
    def get_date(doc=doc):
        spans=[]
        matcher = Matcher(nlp.vocab)
        pattern = [{'IS_DIGIT':True,'OP':'?'},{'IS_TITLE':True,'OP':'?'},{'ORTH':'to','OP':'?'},
                   {'IS_DIGIT':True},{'IS_TITLE':True}, {'IS_DIGIT': True}]
        matcher.add('date', None, pattern)
        matches = matcher(doc)
        for ent_id, start, end in matches:
            if end-start !=4:
                if end-start==5 and str(doc[start]).isdigit() and len(str(doc[start]))<3:
                    span=' '.join([str(doc[start]),str(doc[start+3]),str(doc[start+4])])
                elif end-start==6:
                    span=' '.join([str(doc[start]),str(doc[start+1]),str(doc[end-1])])
                elif end-start==3:
                    span=str(doc[start:end])
                yan=nlp(span)
                span=' '.join([str(yan[0]),str(yan[1]),str(yan[2])[:4]])    
                if datetime.strptime(span, '%d %B %Y').strftime('%Y-%m-%d')>=approval:   
                    spans.append(span)    
        return spans
        
    def get_year(doc=doc):
        spans=[]
        matcher = Matcher(nlp.vocab)
        pattern = [{'IS_TITLE':False}, {'TEXT': {'REGEX':'^\d\d\d\d'}}]
        pattern2=[{'ORTH':'resolution','OP':'!'},{'TEXT': {'REGEX':'^\d\d\d\d'}}]
        matcher.add('year', None, pattern)
        matches = matcher(doc)
        span=[(start,doc[start+1:end]) for ent_id, start, end in matches]
        for i,(s,y) in enumerate(span):
            try:
                if not (int(str(y).strip())<datetime.strptime(approval, '%Y-%m-%d').year or 'resolution'in str(doc[s-1])):
                    spans.append(span[i])
            except:
                pass
        return [x[1] for x in spans]
        
        
    return get_date(),get_year()

In [16]:
TRAIN_DATA = [ 
    ('Recalling section XIV of its resolution 49/233 A of 23 December 1994,', { 
     'entities': [(52,68,'DATE')] 
    }), 
    ('Recalling also its decision 50/500 of 17 September 1996 on the financing of the United Nations Logistics Base at Brindisi, Italy, and its subsequent resolutions thereon, the latest of which was resolution 53/236 of 8 June 1999,', { 
     'entities': [ (38, 55, 'DATE'),(76,109,'ORG'),(215,226,'DATE')] 
    }), 
    ('Having considered the reports of the Secretary-General on the financing of the Logistics Base and the related reports of the Advisory Committee on Administrative and Budgetary Questions,2', { 
     'entities': [(33, 54, 'ORG'),(75,93,'ORG'),(121,185,'ORG')] 
    }), 
    ('Recalling its resolutions 54/196 of 22 December 1999, 55/186 and 55/213 of 20 December 2000 and 55/245 A of 21 March 2001, and decision 1/1 of the Preparatory Committee for the International Conference on Financing for Development,1', { 
     'entities': [(36,52,'DATE'),(108,121,'DATE'),(143,168,'ORG')] 
    }), 
    ('Having considered the reports of the Secretary-General on the financing of the United Nations Observer Mission in Georgia1 and the related report of the Advisory Committee on Administrative and Budgetary Questions,2 ',{
     'entities':[(33,54,'ORG'),(75,110,'ORG'),(149,213,'ORG')]
    }),
    ('Recalling Security Council resolution 854 (1993) of 6 August 1993,',{
    'entities':[(10,26,'ORG'),(52,65,'DATE')]
    }),
    ('the Council established the United Nations Observer Mission in Georgia',{
    'entities':[(24,59,'ORG')]
    }),
    ('Having considered the reports of the Secretary-General on the budget performance of the support account for peacekeeping operations for the period from 1 July 2016 to 30 June 2017,1 on the budget for the support account for peacekeeping operations',{
    'entities':[(33,54,'ORG'),(152,163,'DATE'),(167,179,'DATE'),]
    }),
    ('the report of the Independent Audit Advisory Committee on the proposed budget of the Office of Internal Oversight Services under the support account and the related report of the Advisory Committee on Administrative and Budgetary Questions,5',{
    'entities':[(14,54,'ORG'),(81,122,'ORG'),(175,240,'ORG')] 
    })
        

] 

In [17]:

def main(model=None, output_dir=None, n_iter=150,labels=None):
    if model is not None:
        nlp = spacy.load(model) 
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
    else:
        ner = nlp.get_pipe("ner")
    
    if labels is not None:
        for label in labels:
            ner.add_label(label)

    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    
    move_names = list(ner.move_names)

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            batches = minibatch(TRAIN_DATA, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)


    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])


    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)


        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
            print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    return nlp2

# nlp2=main("en_core_web_sm",r'D:\UN')
# nlp2=main()

In [26]:
def op_to_sentence(op):

    sents=op.text.split('\n')
    sents=[x for x in sents if x is not None and x !=' 'and x!='']
    sentences=[]
    for sent in sents:
        pattern=re.compile('[A-Z][a-z]+')
        match=pattern.search(sent)
        sentence=sent[match.span()[0]:]
        pattern2=re.compile('\s\((.+)\)')
        match2=pattern2.search(sentence)
        if match2 is not None:
            sentence=sentence[:match2.span()[0]]+sentence[match2.span()[1]:]
        sentence=purify(sentence)
        sentence=purify2(sentence)
        sentences.append(sentence)

    return sentences


In [20]:
def purify(sentence):   ##去掉上角标, 去掉resolution, 去掉数字中逗号,去掉in accordance with等
    sentence=sentence.replace('also ','').replace('Also ','').replace('Further ','').replace('further ','')   
    
    begin=sentence.find('in accordance with')
    if begin!=-1:
        comma=sentence[begin:].find(',')
        if comma!=-1:
            sentence=sentence[:begin]+sentence[begin+comma+1:]
        else:
            sentence=sentence[:begin]
            
    begin=sentence.find('in order to')
    if begin!=-1:
        comma=sentence[begin:].find(',')
        if comma!=-1:
            sentence=sentence[:begin]+sentence[begin+comma+1:]
        else:
            sentence=sentence[:begin]
    
    begin=sentence.find('commensurate with')
    if begin!=-1:
        comma=sentence[begin:].find(',')
        if comma!=-1:
            sentence=sentence[:begin]+sentence[begin+comma+1:]
        else:
            sentence=sentence[:begin]
    
    begin=sentence.find('inclusive of')
    if begin!=-1:
        comma=sentence[begin:].find(',')
        if comma!=-1:
            sentence=sentence[:begin]+sentence[begin+comma+1:]
        else:
            sentence=sentence[:begin]
    
    begin=sentence.find('including')
    if begin!=-1:
        comma=sentence[begin:].find(',')
        if comma!=-1:
            sentence=sentence[:begin]+sentence[begin+comma+1:]
        else:
            sentence=sentence[:begin]
            
    begin=sentence.find('as provided for')
    if begin!=-1:
        comma=sentence[begin:].find(',')
        if comma!=-1:
            sentence=sentence[:begin]+sentence[begin+comma+1:]
        else:
            sentence=sentence[:begin]
            
    begin=sentence.find('bearing in mind')
    if begin!=-1:
        comma=sentence[begin:].find(',')
        if comma!=-1:
            sentence=sentence[:begin]+sentence[begin+comma+1:]
        else:
            sentence=sentence[:begin]
            
    begin=sentence.find('in respect of')
    if begin!=-1:
        comma=sentence[begin:].find(',')
        if comma!=-1:
            if len(nlp(sentence[begin:begin+comma]))<12:
                sentence=sentence[:begin]+sentence[begin+comma+1:]
        else:
            if len(nlp(sentence[begin]))<12:
                sentence=sentence[:begin]
    
    begin=sentence.find('as well as')
    if begin!=-1:
        comma=sentence[begin:].find(',')
        if comma!=-1:
            sentence=sentence[:begin]+sentence[begin+comma+1:]
        else:
            sentence=sentence[:begin]
            
    begin=sentence.find('as set out')
    if begin!=-1:
        comma=sentence[begin:].find(',')
        if comma!=-1:
            sentence=sentence[:begin]+sentence[begin+comma+1:]
        else:
            sentence=sentence[:begin]
            
    begin=sentence.find('representing')
    if begin!=-1:
        comma=sentence[begin:].find(',')
        if comma!=-1:
            sentence=sentence[:begin]+sentence[begin+comma+1:]
        else:
            sentence=sentence[:begin]
            
    begin=sentence.find('referred to')
    if begin!=-1:
        comma=sentence[begin:].find(',')
        if comma!=-1:
            sentence=sentence[:begin]+sentence[begin+comma+1:]
        else:
            sentence=sentence[:begin]
    
    index=sentence.rfind(';')
    if index!=-1:
        sentence=sentence[:index+1]
        
    pattern=re.compile('\d+,\d+(,)?(\d+)?(,)?(\d+)?')
    for m in pattern.finditer(sentence):
        mm = m.group()
        sentence = sentence.replace(mm,mm.replace(',',''))

    sentence=nlp(sentence)
    


    matcher = Matcher(nlp.vocab)
    pattern=[{'ORTH':{'IN':['of','in']}},
             {'ORTH':{'IN':['its','General']}},
             {'ORTH':'Assembly','OP':'?'},
             {'ORTH':{'IN':['resolution','resolutions']}}]
    matcher.add('refer',None, pattern)
    matches = matcher(sentence)
    if matches !=[]:
        record=[]
        for _,start,end in matches:
            for item in sentence[end:]:
                if item.is_punct:
                    record.append((end,item.i))
                    break
        sent=[str(t) for t in sentence]
        for t in reversed(record):
            del sent[t[0]:t[1]]
        sentence=' '.join(sent)
        return nlp(sentence)
    
    return sentence
    

In [21]:
def purify2(sentence): # sentence should be nlp format  #去掉 para 去掉 from to 去掉 插入语 合并18.7
    matcher=Matcher(nlp.vocab)
    pattern=[{'ORTH':{'IN':['paragraph','paragraphs']}},{'IS_DIGIT':True},
            {'ORTH':'and'},{'IS_DIGIT':True,'OP':'?'}]
    matcher.add('para',None, pattern)
    matches = matcher(sentence)
    if matches!=[]:
        idx,start,end=matches[0]
        temp=str(sentence[:start+1])+' '+str(sentence[end:])
        sentence=nlp(temp)
    
    matcher=Matcher(nlp.vocab)
    pattern2 = [{'ORTH':'from'},{'IS_DIGIT':True,'OP':'?'},{'IS_TITLE':True,'OP':'?'},{'IS_DIGIT':True,'OP':'?'},{'ORTH':'to'},
               {'IS_DIGIT':True,'OP':'?'},{'IS_TITLE':True}, {'IS_DIGIT': True}]
    matcher.add('date', None, pattern2)
    matches=matcher(sentence)
    if matches!=[]:
        idx,start,end=matches[0]
        temp=str(sentence[:start])+' '+str(sentence[end:])
        sentence=nlp(temp)

    matcher=Matcher(nlp.vocab)
    pattern6 = [{'ORTH':'at'},{'IS_DIGIT':True,'OP':'?'},{'IS_TITLE':True},{'IS_DIGIT':True}]
    matcher.add('date', None, pattern6)
    matches=matcher(sentence)
    if matches!=[]:
        idx,start,end=matches[0]
        temp=str(sentence[:start])+' '+str(sentence[end:])
        sentence=nlp(temp)
        
    record=[]
    matcher=Matcher(nlp.vocab)
    pattern3=[{'ORTH':','},{'OP':'+'},{'ORTH':','}]
    matcher.add('charuyu',None,pattern3)
    matches=matcher(sentence)
    if matches!=[]:
        for _,start,end in matches:
            if end-start<=5:
                record.append((start+1,end))
        if record!=[]:
            sent=[str(t) for t in sentence]
            for t in reversed(record):
                del sent[t[0]:t[1]]
            sentence=nlp(' '.join(sent))
    
    record=[]
    matcher=Matcher(nlp.vocab)
    pattern5=[{'ORTH':','},{'IS_DIGIT':True},{'ORTH':'dollars'}]
    matcher.add('charuyu',None,pattern5)
    matches=matcher(sentence)
    if matches !=[]:
        record=[]
        for _,start,end in matches:
            for item in sentence[end:]:
                if item.is_punct:
                    record.append((start+1,item.i))
                    break
        sent=[str(t) for t in sentence]
        for t in reversed(record):
            del sent[t[0]:t[1]]
        sentence=nlp(' '.join(sent))
    
            
    matcher=Matcher(nlp.vocab)
    pattern4=[{'IS_DIGIT':True},{'ORTH':'.'},{'IS_DIGIT':True}]
    matcher.add('num',None,pattern4)
    matches=matcher(sentence)
    if matches!=[]:
        for _, start, end in matches:
            sentence[start:end].merge()
    
    return sentence
    

In [22]:
def extract_test(sents):
    result=[]
    for sent in sents:
        childs=[]
        grands=[]
        extragrands=[]
        sons=[]
        babies=[]
        roots = [token for token in sent if token.head == token]
        for token in sent:
            if str(token)=='no':
                roots.append(token)
            if str(token)=='not':
                babies.append(token)

        for root in roots:
            for item in root.children:
                if item.dep_!='punct' and item.pos_!='DET' and item.dep_!='aux' and item.dep_!='mark' and item.dep_!='appos':
                    childs.append(item)

            for child in childs:
                for item in child.children:
                    if item.dep_!='punct'and item.pos_!='DET'and item.dep_!='aux'and item.dep_!='mark':
                        if str(item)=='comprising':
                            pass
                        else:
                            grands.append(item)

            for grand in grands:
                for item in grand.children:
                    if item.pos_=='PROPN' or item.pos_=='NOUN'  or item.dep_=='cc'or item.dep_=='ccomp' or item.pos_=='VERB' or item.pos_=='NUM':
                        if str(item)=='period'or str(item)=='amount':
                            grands.remove(grand)
                        elif item.dep_=='advcl':
                            pass
                        else:
                            extragrands.append(item)
                    elif item.pos_=='ADP':
                        for word in item.children:
                            if word.dep_=='pobj':
                                if sent[word.i-1].pos_=='NUM':
                                    sons.append(sent[word.i-1])
                                    sons.append(word)
                                if sent[word.i-1].dep_=='compound' and sent[word.i-2].dep_=='compound':
                                    sons.append(sent[word.i-1])
                                    sons.append(sent[word.i-2])
                                    sons.append(word)
                                    extragrands.append(item)
                                try:
                                    temp=word.nbor().nbor()
                                    if word.is_ancestor(temp) and temp.pos_!='DET':
                                        sons.append(temp)
                                        if temp in temp.head.lefts:
                                            sons.append(temp.head)
                                        else:
                                            sons.append(temp.head.head)
                                        for wd in temp.children:
                                            if wd.dep_=='conj'or wd.dep_=='dobj':
                                                babies.append(wd)
                                except:
                                    pass

                    elif str(item)=='of':
                        for word in item.children:
                            if word.pos_=='NOUN':
                                extragrands.append(item)
                                sons.append(word)
                                for wd in word.children:
                                    if wd.dep_=='conj':
                                        sons.append(wd)
                    elif str(item)=='for':
                        for word in item.children:
                            if word.pos_=='NUM':
                                extragrands.append(item)
                                sons.append(word)

            for extragrand in extragrands:
                for item in extragrand.children:
                    if item.pos_=='PROPN' or item.pos_=='NOUN' or item.dep_=='nummod' or item.pos_=='NUM':
                        sons.append(item)
                    if item.pos_=='VERB':
                        for word in item.children:
                            if word.pos_=='NOUN':
                                sons.append(item)
                                babies.append(word)
                    if item.dep_=='amod':
                        if item in list(extragrand.lefts):
                            sons.append(item)
                    if str(item)=='of':
                        for word in item.children:
                            if word.pos_=='NOUN'and str(word)!='dollars':
                                sons.append(item)
                                babies.append(word)
                                for wd in word.children:
                                    if wd.dep_=='conj':
                                        babies.append(wd)
                    if str(item)=='on':
                        for word in item.children:
                            if word.pos_=='NOUN':
                                sons.append(item)
                                babies.append(word)
                                for wd in word.children:
                                    if wd.dep=='conj' or wd.dep_=='amod' and wd in list(word.lefts):
                                        babies.append(wd)


        dic={}
        for t in roots+childs+grands+extragrands+sons+babies:
            dic[t]=t.i

        trunk=sorted(dic.items(), key=lambda x: x[1])
        extracted=[t[0] for t in trunk]
    
        result.append(extracted)
    return result


In [449]:

# displacy.render(sents[-5], style='dep', jupyter=True, options={'distance': 70})

    

In [460]:

# childs=[]
# grands=[]
# extragrands=[]
# sons=[]
# daugh=[]
# babies=[]
# sent=sents[-5]
# roots = [token for token in sent if token.head == token]

# for token in sent:
#     if str(token)=='no':
#         roots.append(token)
#     if str(token)=='not':
#         babies.append(token)
# print("roots:",roots)
        
# for root in roots:
#     for item in root.children:
#         if item.dep_!='punct' and item.pos_!='DET' and item.dep_!='aux' and item.dep_!='mark' and item.dep_!='appos':
#             childs.append(item)
#     print("childs:",childs)
    
#     for child in childs:
#         for item in child.children:
#             if item.dep_!='punct'and item.pos_!='DET'and item.dep_!='aux'and item.dep_!='mark':
#                 if str(item)=='comprising':
#                     pass
#                 else:
#                     grands.append(item)
#     print("grands:",grands)
    
#     for grand in grands:
#         for item in grand.children:
#             if item.pos_=='PROPN' or item.pos_=='NOUN'  or item.dep_=='cc' or item.dep_=='ccomp' or item.pos_=='VERB' or item.pos_=='NUM':
#                 if str(item)=='period' or str(item)=='amount' :
#                     grands.remove(grand)
#                 elif item.dep_=='advcl':
#                     pass
#                 elif item.pos_=='VERB' and item in list(grand.lefts):
#                     pass

#                 else:
#                     extragrands.append(item)
                    
#             elif item.pos_=='ADP':
#                 for word in item.children:
#                     if word.dep_=='pobj':   
#                         if sent[word.i-1].pos_=='NUM':
#                             print(666)
#                             daugh.append(sent[word.i-1])
#                             daugh.append(word)
#                         if sent[word.i-1].dep_=='compound' and sent[word.i-2].dep_=='compound':
#                             print(777)
#                             daugh.append(sent[word.i-1])
#                             daugh.append(sent[word.i-2])
#                             daugh.append(word)
#                             extragrands.append(item)
#                         temp=word.nbor().nbor()
# #                         if word.is_ancestor(temp) and temp.pos_=='NUM':
# #                             sons.append(temp)
# #                             sons.append(temp.head)
#                         if word.is_ancestor(temp) and temp.pos_!='DET':
#                             print(888)
#                             print(word,temp)
#                             daugh.append(temp)
#                             if temp in temp.head.lefts:
#                                 daugh.append(temp.head)
#                             else:
#                                 daugh.append(temp.head.head)
#                             for wd in temp.children:
#                                 if wd.dep_=='conj' or wd.dep_=='dobj':
#                                     daugh.append(wd)

#             elif str(item)=='of':
#                 for word in item.children:
#                     if word.pos_=='NOUN':
#                         extragrands.append(item)
#                         sons.append(word)
#                         for wd in word.children:
#                             if wd.dep_=='conj':
#                                 sons.append(wd)
#             elif str(item)=='for':
#                 for word in item.children:
#                     if word.pos_=='NUM':
#                         extragrands.append(item)
#                         sons.append(word)
            
            
#     print("extragrands:",extragrands)
    
#     for extragrand in extragrands:
#         for item in extragrand.children:
#             if item.pos_=='PROPN' or item.pos_=='NOUN' or item.dep_=='nummod' or item.dep_=='pobj' or item.pos_=='NUM':
#                 sons.append(item)
#             if item.pos_=='VERB':
#                 for word in item.children:
#                     if word.pos_=='NOUN':
#                         sons.append(item)
#                         babies.append(word)
#             if item.dep_=='amod':
#                 print(list(extragrand.lefts))
#                 if item in list(extragrand.lefts):
#                     sons.append(item)
#             if str(item)=='of':
#                 for word in item.children:
#                     if word.pos_=='NOUN'and str(word)!='dollars':
#                         sons.append(item)
#                         babies.append(word)
#                         for wd in word.children:
#                             if wd.dep_=='conj':
#                                 babies.append(wd)
#             if str(item)=='on':
#                 for word in item.children:
#                     if word.pos_=='NOUN':
#                         sons.append(item)
#                         babies.append(word)
#                         for wd in word.children:
#                             if wd.dep=='conj' or wd.dep_=='amod' and wd in list(word.lefts):
#                                 babies.append(wd)
                
#     print("sons:",sons)
#     print("daugh:",daugh)
#     print("babies:",babies)

# dic={}
# for t in roots+childs+grands+extragrands+sons+babies+daugh:
#     dic[t]=t.i

# trunk=sorted(dic.items(), key=lambda x: x[1])

# print([t[0] for t in trunk])

roots: [Decides]
childs: [added]
grands: [increase, be, to, from]
666
777
extragrands: [in, credits]
sons: [income]
daugh: [37400, dollars, assessment, staff, income]
babies: []
[Decides, increase, 37400, dollars, in, staff, assessment, income, be, added, to, credits]


In [411]:
# for token in sents[3]:
#     print(token,'\t',token.pos_,'\t',token.dep_,'\t',token.head)

Expresses 	 NOUN 	 amod 	 concern
concern 	 NOUN 	 ROOT 	 concern
at 	 ADP 	 prep 	 concern
the 	 DET 	 det 	 delay
delay 	 NOUN 	 pobj 	 at
experienced 	 VERB 	 acl 	 delay
by 	 ADP 	 agent 	 experienced
the 	 DET 	 det 	 Secretary-General
Secretary-General 	 PROPN 	 pobj 	 by
in 	 ADP 	 prep 	 experienced
deploying 	 VERB 	 pcomp 	 in
and 	 CCONJ 	 cc 	 deploying
providing 	 VERB 	 conj 	 deploying
adequate 	 ADJ 	 amod 	 resources
resources 	 NOUN 	 dobj 	 providing
to 	 ADP 	 prep 	 providing
some 	 DET 	 det 	 missions
recent 	 ADJ 	 amod 	 missions
peacekeeping 	 NOUN 	 compound 	 missions
missions 	 NOUN 	 pobj 	 to
, 	 PUNCT 	 punct 	 concern
in 	 ADP 	 prep 	 those
particular 	 ADJ 	 amod 	 in
those 	 DET 	 appos 	 concern
in 	 ADP 	 prep 	 those
Africa 	 PROPN 	 pobj 	 in
; 	 PUNCT 	 punct 	 concern
